In [20]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# Step 1: Data Preparation
# Load the dataset
data = pd.read_csv("C:/Users/Michael.A_Sydani/Desktop/test_folder2/ds_task_2/orders.csv.csv")
data.head(5)

,order_id,order_dow,order_hour_of_day,days_since_prior_order,air fresheners candles,asian foods,baby accessories,baby bath body care,baby food formula,bakery desserts,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
0,1597,1,8,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2011,4,10,30,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2822,0,8,29,0,0,0,0,1,0,...,0,0,0,2,0,0,0,0,0,2
3,2889,1,15,8,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,3971,2,18,8,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# Step 2: Exploratory Data Analysis (EDA)
# Check the structure and summary statistics of the dataset
print(data.head())
print(data.describe())

   order_id  order_dow  order_hour_of_day  days_since_prior_order  \
0      1597          1                  8                       4   
1      2011          4                 10                      30   
2      2822          0                  8                      29   
3      2889          1                 15                       8   
4      3971          2                 18                       8   

   air fresheners candles  asian foods  baby accessories  baby bath body care  \
0                       0            0                 0                    0   
1                       0            0                 0                    0   
2                       0            0                 0                    0   
3                       0            0                 0                    0   
4                       0            0                 0                    0   

   baby food formula  bakery desserts  ...  spreads  tea  \
0                  0                0 

In [22]:

# Clean and format the dataset as needed
# Handle missing values, encode categorical variables, etc.
# For example:
# Handle missing values
data.dropna(inplace=True)

# Encode categorical variables (if any)
# For example, if 'gender' is represented as strings:
# data['gender'] = data['gender'].map({'male': 0, 'female': 1})

# Remove non-item columns
item_data = data.drop(['order_dow', 'order_hour_of_day', 'days_since_prior_order'], axis=1)

In [23]:

from sklearn.cluster import KMeans

# Apply K-means clustering
kmeans = KMeans(n_clusters=2)  # Assuming we want to identify two clusters
kmeans.fit(item_data)

# Get cluster labels
cluster_labels = kmeans.labels_

# Add cluster labels to the original dataset
data['cluster'] = cluster_labels

In [24]:
!pip install mlxtend

In [25]:

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Convert binary item data to transaction format
# Each row represents a transaction, and items are encoded as binary values
transaction_data = item_data.applymap(lambda x: 1 if x > 0 else 0)

# Find frequent itemsets using Apriori algorithm
frequent_itemsets = apriori(transaction_data, min_support=0.05, use_colnames=True)

# Find association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)


C:\Users\Michael.A_Sydani\AppData\Local\Temp\ipykernel_23436\606083003.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  transaction_data = item_data.applymap(lambda x: 1 if x > 0 else 0)
C:\Users\Michael.A_Sydani\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [26]:

# Analyze clusters and association rules to identify patterns
# Explore clusters to see if there are any distinct item preferences
cluster_analysis = data.groupby('cluster').mean()

# Analyze association rules to see if certain items tend to be purchased together
# You can filter rules based on lift, confidence, etc., to find meaningful patterns
significant_rules = rules[(rules['lift'] > 1) & (rules['confidence'] > 0.5)]


In [27]:

def predict_gender(order_id):
    # Assuming 'data' contains the dataset with 'order_id' and 'cluster' columns
    # Find the cluster label associated with the given order_id
    cluster_label = data[data['order_id'] == order_id]['cluster'].values[0]
    
    # Determine the gender based on the cluster label
    if cluster_label == 0:
        predicted_gender = "Male"
    else:
        predicted_gender = "Female"
    
    return predicted_gender

In [30]:

# Assuming 'data' contains the dataset with 'order_id' and 'cluster' columns
# Print order IDs along with their associated gender clusters
for cluster_label in data['cluster'].unique():
    cluster_data = data[data['cluster'] == cluster_label]
    gender = "Male" if cluster_label == 0 else "Female"
    print("Gender Cluster:", gender)
    print("Order IDs:", cluster_data['order_id'].values)

Gender Cluster: Female
Order IDs: [   1597    2011    2822 ... 1702165 1702465 1703129]
Gender Cluster: Male
Order IDs: [1703666 1705054 1705426 ... 3420769 3420894 3420909]


In [31]:

order_id = 3420909  # Replace with the desired order_id
predicted_gender = predict_gender(order_id)
print("Predicted gender for order", order_id, "is:", predicted_gender)

Predicted gender for order 3420909 is: Male
